In [149]:
import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive')

mapping_name_coords = pd.read_csv('/content/drive/MyDrive/WDL/mapping_name_coords.csv', sep=',')

mapping_name_coords['randNumCol'] = np.random.randint(0, 5000, mapping_name_coords.shape[0])/5000

mapping_name_coords['coordinates'] = mapping_name_coords['coordinates'].str.replace('(', '').str.replace(')', '').str.replace('\'', '')
mapping_name_coords[['latitude','longitude']] = mapping_name_coords.coordinates.str.split(",",expand=True,).astype(float)

mapping_name_coords

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,name,coordinates,freguesia,concelho,cidade,randNumCol,latitude,longitude
0,0,image0,"38.73277225961562, -9.172048739336",Campolide,Lisboa,Lisboa,0.8456,38.732772,-9.172049
1,1,image1,"38.76039827177381, -9.190202095966573",Carnide,Lisboa,Lisboa,0.7308,38.760398,-9.190202
2,2,image10,"38.7439627, -9.1136023",Marvila,Lisboa,Lisboa,0.4260,38.743963,-9.113602
3,3,image100,"38.77974496618023, -9.124785817496566",Olivais,Lisboa,Lisboa,0.8942,38.779745,-9.124786
4,4,image1000,"38.7754189, -9.1708593",Lumiar,Lisboa,Lisboa,0.4792,38.775419,-9.170859
...,...,...,...,...,...,...,...,...,...
2495,2495,votingImage995,"38.73844792547146, -9.136083471008174",None,Lisboa,Lisboa,0.9744,38.738448,-9.136083
2496,2496,votingImage996,"38.740054, -9.1913993",Benfica,Lisboa,Lisboa,0.0892,38.740054,-9.191399
2497,2497,votingImage997,"38.7626013, -9.1300163",São João de Brito,Lisboa,Lisboa,0.4674,38.762601,-9.130016
2498,2498,votingImage998,"38.7679489, -9.161832993873306",Lumiar,Lisboa,Lisboa,0.0416,38.767949,-9.161833


In [158]:
import os
from ipywidgets import Image, Layout

image_data = {}
for img_filename in os.listdir('/content/drive/MyDrive/WDL/images'):
    image_name = img_filename.split('angle')[0]
    with open(f"/content/drive/MyDrive/WDL/images/{img_filename}", "rb") as f:
        b = f.read()
        image_data[image_name] = b

In [159]:
import plotly.graph_objs as go

fig = go.FigureWidget(
    data=[
        dict(
            type='scattergl',
            x=mapping_name_coords['latitude'],
            y=mapping_name_coords['longitude'],
            mode='markers',
            marker=dict(color=mapping_name_coords['randNumCol'],showscale=True)
        )
    ],
)

fig.layout.xaxis.title = 'Latitude'
fig.layout.yaxis.title = 'Longitude'

fig.show()

In [162]:
scatter = fig.data[0]
scatter.marker.opacity = 1
scatter.marker.size = 4
scatter.text = mapping_name_coords['name']
scatter.hoverinfo = 'text'

fig.show()

In [163]:
def set_opacity(opacity, size):
    scatter.marker.opacity = opacity
    scatter.marker.size = size


from ipywidgets import interactive
opacity_slider = interactive(set_opacity,
                             opacity=(0.0, 1.0, 0.01),
                             size=(1, 10, 0.25))
opacity_slider

opacity_slider.children[0].layout.width = '400px'
opacity_slider.children[1].layout.width = '400px'

In [164]:
from ipywidgets import HTML
details = HTML()
details

HTML(value='')

In [165]:
def hover_fn(trace, points, state):
    ind = points.point_inds[0]
    details.value = mapping_name_coords.iloc[ind].to_frame().to_html()

scatter.on_hover(hover_fn)

In [166]:
from ipywidgets import Image, Layout
image_widget = Image(
    value=image_data['image0'],
    layout=Layout(height='252px', width='400px')
)
image_widget

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [167]:
def hover_fn(trace, points, state):

    ind = points.point_inds[0]

    # Update details HTML widget
    details.value = mapping_name_coords.iloc[ind].to_frame().to_html()

    # Update image widget
    image_name = mapping_name_coords['image'][ind]+'_name'
    image_widget.value = image_data[image_name]

scatter.on_hover(hover_fn)

In [168]:
from ipywidgets import HBox, VBox

VBox([fig,
      opacity_slider,
      HBox([image_widget, details])])

    'data': [{'hoverinfo': 'text',
              'marker': {'color': array([0.84…